In [ ]:
import os
import time
import pandas as pd

import torch
import torch.nn as nn

In [ ]:
from transformers import (
    AutoConfig,
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
)
from transformers import (
    pipeline, 
    TrainingArguments, 
    Trainer
)
from peft import ( 
		LoraConfig, 
		PeftConfig, 
		PeftModel, 
		get_peft_model, 
		prepare_model_for_kbit_training
)

In [ ]:
MODEL_NAME = "/kaggle/input/llama-3/transformers/8b-hf/1"

new_model = "/kaggle/input/llama_1_try/transformers/v1/1"

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
def test_input(prompt_i, text_i, prompt_i2):
    
    prompt = prompt_i + text_i + prompt_i2
    inputs = tokenizer(prompt, return_tensors="pt").input_ids
    
    input_ids = inputs.to('cuda')
    outputs = model.generate(input_ids, max_new_tokens=120, do_sample=False, pad_token_id=tokenizer.eos_token_id)
    return(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
df = pd.read_csv("/kaggle/input/testing-data/test_data.csv")

In [ ]:
outputs = []
p1 = "Напиши резюме тексту: "

for ind,row in df[:100].iterrows():
    
    input_text = row["text"]
    res = test_input(p1, input_text, "")
    outputs.append((res,row["post_id"], input_text))
    
smdf = pd.DataFrame(outputs)
smdf.to_csv("test_df_llama1.csv")